# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources in the README.md file
- Happy learning!

# Challenge 1 - Accessing our database
Create a connection to access the sakila database with the ip, user and password provided in class. Take a look at the data. Do some joins in order to have more elaborated tables in a dataframe (you can try first the query in the DBMS and then use it in your notebook).


In [2]:
from sqlalchemy import create_engine
import pandas as pd
import pymysql
import sqlalchemy as db

db_path = '/home/emily/Downloads/juliarochflores.db'

conn_str = f'sqlite:///{db_path}'

engine = create_engine(conn_str)
connection = engine.connect()
metadata = db.MetaData()

In [6]:
data = pd.read_sql_query("""
SELECT personal_info.id,
name,
age,
gender,
country,
"position",
personal_info."Unnamed: 0",
image,
"Source",
worth,
worthChange,
realTimePosition
FROM personal_info
JOIN rank_info
ON personal_info.id = rank_info.id
JOIN business_info
ON personal_info.id = business_info.id;
""", engine)
data

,id,name,age,gender,country,position,Unnamed: 0,image,Source,worth,worthChange,realTimePosition
0,8760,jefF BEZOs,54 years old,None,United States,1.0,52,https://specials-images.forbesimg.com/imageser...,Technology ==> Amazon,112.0 BUSD,0.0 millions USD,1
1,7500,bILL GATES,62 years old,Male,None,2.0,53,https://specials-images.forbesimg.com/imageser...,Technology ==> Microsoft,90.0 BUSD,-0.001 millions USD,2
2,5751,WARREN BUFFEtt,87 years old,M,None,3.0,54,https://specials-images.forbesimg.com/imageser...,Finance and Investments ==> Berkshire Hathaway,84.0 BUSD,-0.002 millions USD,3
3,8365,bernARD ARNAULT,69 years old,None,France,4.0,55,https://specials-images.forbesimg.com/imageser...,Fashion & Retail ==> LVMH,72.0 BUSD,0.0 millions USD,4
4,5328,maRK ZUCKERBERG,1985,None,United States,5.0,56,https://specials-images.forbesimg.com/imageser...,Technology ==> Facebook,71.0 BUSD,0.0 millions USD,5
...,...,...,...,...,...,...,...,...,...,...,...,...
2203,4506,zhao XIAOQiang,51 years old,M,China,2204.0,2255,https://specials-images.forbesimg.com/imageser...,"Fashion & Retail ==> fashion, entertainment",1.0 BUSD,0.0 millions USD,2134
2204,6986,zHOU LIANgzhang,55 years old,None,None,2205.0,2256,https://specials-images.forbesimg.com/imageser...,Manufacturing ==> electrical equipment,1.0 BUSD,nan millions USD,2134
2205,1828,zhu XINGMIng,51 years old,M,People's Republic of China,2206.0,2257,https://specials-images.forbesimg.com/imageser...,Manufacturing ==> electrical equipment,1.0 BUSD,0.0 millions USD,1856
2206,6448,ZHUO JUN,52 years old,Female,None,2207.0,2258,https://specials-images.forbesimg.com/imageser...,Manufacturing ==> printed circuit boards,1.0 BUSD,0.0 millions USD,1978


array(['jefF BEZOs', 'bILL GATES', 'WARREN BUFFEtt', ..., 'zhu XINGMIng',
       'ZHUO JUN', 'olguN ZOrlu'], dtype=object)

# Challenge 2 - Working with JSON files

Import the pandas library.

In [7]:
import pandas as pd

In the next cell, load the data in `nasa.json` in the `data` folder and load it into a pandas dataframe. Name the dataframe `nasa`.

In [14]:
nasa = pd.read_json('/home/emily/Bootcamp/dataptmad1120/module-1/importing-exporting-data/data/nasa.json', orient='records')

Now that we have loaded the data, let's examine it using the `head()` function.

In [15]:
nasa.head()

,:@computed_region_cbhk_fwbd,:@computed_region_nnqa_25f4,fall,geolocation,id,mass,name,nametype,recclass,reclat,reclong,year
0,NaN,NaN,Fell,"{'type': 'Point', 'coordinates': [6.08333, 50....",1,21.0,Aachen,Valid,L5,50.77500,6.08333,1880-01-01T00:00:00.000
1,NaN,NaN,Fell,"{'type': 'Point', 'coordinates': [10.23333, 56...",2,720.0,Aarhus,Valid,H6,56.18333,10.23333,1951-01-01T00:00:00.000
2,NaN,NaN,Fell,"{'type': 'Point', 'coordinates': [-113, 54.216...",6,107000.0,Abee,Valid,EH4,54.21667,-113.00000,1952-01-01T00:00:00.000
3,NaN,NaN,Fell,"{'type': 'Point', 'coordinates': [-99.9, 16.88...",10,1914.0,Acapulco,Valid,Acapulcoite,16.88333,-99.90000,1976-01-01T00:00:00.000
4,NaN,NaN,Fell,"{'type': 'Point', 'coordinates': [-64.95, -33....",370,780.0,Achiras,Valid,L6,-33.16667,-64.95000,1902-01-01T00:00:00.000


#### The `value_counts()` function is commonly used in pandas to find the frequency of every value in a column.

In the cell below, use the `value_counts()` function to determine the frequency of all types of asteroid landings by applying the function to the `fall` column.

In [5]:
# your code here

Finally, let's save the dataframe as a json file again. Save the dataframe using the `orient=records` argument and name the file `nasa-output.json`. Remember to save the file inside the `data` folder.

In [6]:
# your code here

# Challenge 2 - Working with CSV and Other Separated Files

CSV files are more commonly used as dataframes. In the cell below, load the file from the URL provided using the `read_csv()` function in pandas. Starting version 0.19 of pandas, you can load a CSV file into a dataframe directly from a URL without having to load the file first and then transform it. The dataset we will be using contains information about NASA shuttles.

In the cell below, we define the column names and the URL of the data. Following this cell, read the tst file to a variable called `shuttle`. Since the file does not contain the column names, you must add them yourself using the column names declared in `cols` using the `names` argument. Additionally, a tst file is space separated, make sure you pass ` sep=' '` to the function.

In [1]:
cols = ['time', 'rad_flow', 'fpv_close', 'fpv_open', 'high', 'bypass', 'bpv_close', 'bpv_open', 'class']
tst_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/shuttle/shuttle.tst'

In [8]:
# your code here

Let's verify that this worked by looking at the `head()` function.

In [9]:
# your code here

To make life easier for us, let's turn this dataframe into a comma separated file by saving it using the `to_csv()` function. Save `shuttle` into the file `shuttle.csv` and ensure the file is comma separated, that we are not saving the index column and that the file is saved inside the `data` folder.

In [10]:
# your code here

# Challenge 3 - Working with Excel Files

We can also use pandas to convert excel spreadsheets to dataframes. Let's use the `read_excel()` function. In this case, `astronauts.xls` is in the `data` folder. Read this file into a variable called `astronaut`. 

Note: Make sure to install the `xlrd` library if it is not yet installed.

In [5]:
!conda install -y xlrd

Solving environment: done

## Package Plan ##

  environment location: /home/emily/anaconda3/envs/ironhack_env

  added / updated specs:
    - xlrd


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    xlrd-2.0.1                 |     pyhd3eb1b0_0          90 KB
    ------------------------------------------------------------
                                           Total:          90 KB

The following packages will be UPDATED:

  xlrd                                           1.2.0-py_0 --> 2.0.1-pyhd3eb1b0_0



xlrd-2.0.1           | 90 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


Use the `head()` function to inspect the dataframe.

In [13]:
df_excel = pd.read_excel('data/astronauts.xls')

df_excel.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/astronauts.xls'

Use the `value_counts()` function to find the most popular undergraduate major among all astronauts.

In [13]:
# your code here

Due to all the commas present in the cells of this file, let's save it as a tab separated csv file. In the cell below, save `astronaut` as a **tab separated file** using the `to_csv` function. Call the file `astronaut.csv`. Remember to remove the index column and save the file in the `data` folder.

In [14]:
# your code here

# Bonus Challenge - Fertility Dataset

Visit the following [URL](https://archive.ics.uci.edu/ml/datasets/Fertility) and retrieve the dataset as well as the column headers. Determine the correct separator and read the file into a variable called `fertility`. Examine the dataframe using the `head()` function. 

Look in Google for a way to retrieve this data!

In [15]:
# your code here